# 事前の設定（GPUの設定）
* 上のメニュー「ランタイム」 -> ランタイムのタイプを変更」を選択する
* ハードウェアアクセラレータを「GPU」に変更し，保存する

In [0]:
!nvidia-smi    # GPUの確認, '!'を先頭につけるとシェルコマンドになる

# Pytorch（Deep Learningのフレームワーク）のインストール
* http://pytorch.org/

In [0]:
!pip install torch torchvision

# PILの再インストール

In [0]:
!pip install pillow==4.1.1

# GPUが使えているかの確認

In [0]:
import torch
torch.cuda.is_available()

# ImageNet 学習済みモデルによる画像認識
* https://pytorch.org/docs/stable/torchvision/models.html

In [0]:
import torchvision.models as models
cnn_model = models.vgg19(pretrained=True)    # 学習済みモデルの読み込み
cnn_model.eval()    # 評価モードにする

# ラベル名の取得

In [0]:
!wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

In [0]:
import json
class_index = json.load(open('imagenet_class_index.json', 'r'))
labels = {int(key):value for (key, value) in class_index.items()}

# 予測のための関数の定義

In [0]:
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def predict(cnn_model, labels, image_file):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
    preprocess = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224), transforms.ToTensor(), normalize])

    print('Read ' + image_file)
    img = Image.open(image_file, 'r')
    
    # exif情報を使った画像回転の補正
    if hasattr(img, '_getexif'):
        _orientation = img._getexif()[274]
        if _orientation == 6:
            img = img.rotate(-90, expand=True)
        elif _orientation == 8:
            img = img.rotate(-270, expand=True)
        elif _orientation == 3:
            img = img.rotate(180, expand=True)
    
    # 画像の前処理
    img = img.convert('RGB')
    img_tensor = preprocess(img)
    img_tensor.unsqueeze_(0)

    # 畳み込みニューラルネットワークの実行
    out = cnn_model(Variable(img_tensor))
    
    out = nn.functional.softmax(out, dim=1)    # 出力を確率にする
    out = out.data[0].numpy()

    rank_id = np.argsort(out)[::-1][:5]

    for i in range(5):
        print('%d: '%(i) + labels[rank_id[i]][1] + ' (' + str(out[rank_id[i]]) + ')')
    
    plt.imshow(np.array(img))
    plt.grid(False)
    plt.tick_params(labelbottom="off",bottom="off")    # x軸の削除
    plt.tick_params(labelleft="off",left="off")    # y軸の削除
    #print(np.array(img).shape)
    
    return

# 画像ファイルのアップロード
* 「ファイルの選択」から画像ファイルをアップロードする

In [0]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# 画像認識の実行

In [0]:
predict(cnn_model, labels, file_name)